In [8]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import os
import sys


###########################################################################################

#import point cloud

#pcd_01 = o3d.io.read_point_cloud("C:/Users/User/Open3D_Folder/Data/model_clean.ply")
pcd_01 = o3d.io.read_point_cloud("C:/Users/User/Open3D_Folder/Data/test_torso_01.ply")

#pcd_02.paint_uniform_color([2, 0.706, 0])
#o3d.visualization.draw_geometries([pcd_01, pcd_02])

###########################################################################################
#import mesh

#path = ("C:/Users/User/Open3D_Folder/Data/test_torso_01.ply")

#torso_mesh = o3d.data.TorsoMesh()
#mesh = o3d.io.read_triangle_mesh(path)
#mesh.paint_uniform_color([2, 0.306, 0])
#print(mesh)
#print('Vertices:')
#print(np.asarray(mesh.vertices))
#print('Triangles:')
#print(np.asarray(mesh.triangles))

#print("Try to render a mesh with normals (exist: " +
#      str(mesh.has_vertex_normals()) + ") and colors (exist: " +
#      str(mesh.has_vertex_colors()) + ")")
#o3d.visualization.draw_geometries([mesh])

#print("Computing normal and rendering it.")
#mesh.compute_vertex_normals()
#print(np.asarray(mesh.triangle_normals))
#o3d.visualization.draw_geometries([mesh])


###########################################################################################
#oriented bounding box

#obj1 = pcd_01.get_axis_aligned_bounding_box()
#obj1.color = (1, 0, 0)
#obj2 = pcd_01.get_oriented_bounding_box()
#obj2.color = (0, 0, 1)
#o3d.visualization.draw_geometries([pcd_01, obj1, obj2])

##########################################################################################
#calculate centre of point cloud
##########################################################################################

pcd_points_01 = np.asarray(pcd_01.points)
pcd_centre_01 = pcd_points_01[1,:] + pcd_points_01[2,:]
pcd_centre_01[0] = 0
pcd_centre_01[1] = 0
pcd_centre_01[2] = 0

#calculate centre of torso point cloud####################################################

nr = len(pcd_points_01)
print(nr)

#calculate centre
#---------------------
for i in range(nr):
  pcd_centre_01 = pcd_centre_01 + pcd_points_01[i,:]
#---------------------
pcd_centre_01 = pcd_centre_01/nr

#centre point cloud
#---------------------
for i in range(nr):
  pcd_points_01[i,:] = pcd_points_01[i,:] - pcd_centre_01

print(np.asarray(pcd_points_01)[0,:])

##########################################################################################
#make copy of point cloud and perform reflection about sagittal plane
##########################################################################################

#make copy of torso object
#pcd_points_01_reflect = pcd_points_01
pcd_points_01_reflect = pcd_points_01.copy()
#print(np.asarray(pcd_centre_01)[:])
    #source_temp = copy.deepcopy(source)
    #target_temp = copy.deepcopy(target)

#do reflection about sagittal plane
for i in range(nr):
  temp = pcd_points_01[i,0]
  pcd_points_01_reflect[i,0] = -temp

print(np.asarray(pcd_points_01)[0,:])
print(np.asarray(pcd_points_01_reflect)[0,:])

#------------------------------------------------------------------------------------------
#Detect which points of the reflection are positioned inside the original volume
#------------------------------------------------------------------------------------------

#Variablennamen ueberpruefen

sign_dist = pcd_points_01[:,0].copy()
sign_dist_metric = pcd_points_01[:,0].copy()

for i in range(nr):
    distmin = 100000

    #Distanzfunktion von Open3D nutzen anstatt Schleife
    for k in range(math.floor(nr/10)):
        
        j = k*10
#        #calculate manually the distance between original and reflection
        dists_man = math.sqrt( (pcd_points_01[i,0] - pcd_points_01_reflect[j,0])**2 + (pcd_points_01[i,1] - pcd_points_01_reflect[j,1])**2 + (pcd_points_01[i,2] - pcd_points_01_reflect[j,2])**2 )

        if dists_man < distmin: 
            distmin = dists_man
            near_point = pcd_points_01_reflect[j,:]
            sign_dist_metric[i] = dists_man

    dist_orig = math.sqrt( (pcd_points_01[i,0] - pcd_centre_01[0])**2 + (pcd_points_01[i,1] - pcd_centre_01[1])**2 + (pcd_points_01[i,2] - pcd_centre_01[2])**2 )
    dist_refl = math.sqrt( (near_point[0] - pcd_centre_01[0])**2 + (near_point[1] - pcd_centre_01[1])**2 + (near_point[2] - pcd_centre_01[2])**2 )

    if dist_orig > dist_refl:
        sign_dist[i] = 1
    else: 
        sign_dist[i] = 0
        sign_dist_metric[i] = - sign_dist_metric[i]

print("Print the signed distances of the first 10 points")
print(np.asarray(sign_dist)[:])

#------------------------------------------------------------------------------------------

#Assign manipulated point-coordinates to new o3d objects
pointSet1 = o3d.geometry.PointCloud()
pointSet1.points = o3d.utility.Vector3dVector(pcd_points_01)
pointSet2 = o3d.geometry.PointCloud()
pointSet2.points = o3d.utility.Vector3dVector(pcd_points_01_reflect)

pointSet1.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1, max_nn=30))
pointSet2.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1, max_nn=30))

#plot original and reflection
pointSet1.paint_uniform_color([2, 0.706, 0])
pointSet2.paint_uniform_color([2, 0.206, 0])
#o3d.visualization.draw_geometries([pointSet1, pointSet2])

###########################################################################################
#ICP matching for original and reflection
###########################################################################################

###########################################################################################

#type(source)

#------------------------------------------------------------------------
def draw_registration_result_init(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


#------------------------------------------------------------------------
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

    calculate_asymmetry_dist(source_temp, target_temp, sign_dist_metric)


#--------------------------------------------------------------------------
def point_to_point_icp(source, target, threshold, trans_init):
    print("Apply point-to-point ICP")
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())

    print(reg_p2p)
    print("Transformation is:")
    print(reg_p2p.transformation, "\n")
    draw_registration_result_init(source, target, reg_p2p.transformation)

#-------------------------------------------------------------------------
def point_to_plane_icp(source, target, threshold, trans_init):
    print("Apply point-to-plane ICP")
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())

    print(reg_p2l)
    print("Transformation is:")
    print(reg_p2l.transformation, "\n")
    draw_registration_result(source, target, reg_p2l.transformation)

#---------------------------------------------------------------------------
###########################################################################################
#calculate distance between original and reflected torso
###########################################################################################

def calculate_asymmetry_dist(source_temp, target_temp, sign_dist_metric):
    #o3d.visualization.draw_geometries([source_temp, target_temp])

    #calculate distance between the 2 torso's
    dists = source_temp.compute_point_cloud_distance(target_temp)
    dists = np.asarray(dists)
    dists_norm = copy.deepcopy(dists) 

    #Paint point color according to distance---------------------
    lendists = len(dists)
    maxdist = max(dists)
    np_colors = np.random.rand(lendists, 3)  #create 3d float array for colors

    #assign colors normalized to 1 to RGB color array
    for i in range(0, lendists):
        #if sign_dist_metric[i] > 0:
            dists_norm[i] = dists[i]/maxdist
            np_colors[i,0] = dists_norm[i]
            np_colors[i,1] = 1-dists_norm[i]
            np_colors[i,2] = 1-dists_norm[i]
        #elif sign_dist_metric[i] < 0:
        #    np_colors[i,0] = 0
        #    np_colors[i,1] = 0
        #    np_colors[i,2] = 0

    #create temporary PointCloud pcd for visualization and assign points as well as colors 
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(source_temp.points)
    pcd.colors = o3d.utility.Vector3dVector(np_colors)
    o3d.visualization.draw_geometries([pcd])

    #------------------------------------------------------------
    #Print Original and PointCloud with removed points (>9mm)
    #------------------------------------------------------------

    ind = np.where(sign_dist_metric > 10)[0]
    pcd_threshold = source_temp.select_by_index(ind)
    pcd_threshold.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([pcd_threshold, source_temp])

    print("Print the distances of the first 10 points")
    print(np.asarray(dists)[:10])

    #---------------------------------------------
    #Save all asymmetry distances in text file
    #---------------------------------------------
    print('Save distances in float array')
    #---------------------------------------------
    lendists = len(dists)
    f = open("distances.dat", "w+")
    for i in range(0, lendists):
        f.write(str(sign_dist_metric[i]))
        f.write("\n")  
    #----------------------------------------------
    
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------

#-----------------------------------------------------------------------------------
#Main part for ICP registration utilizting the prior defined functions
#-----------------------------------------------------------------------------------

if __name__ == "__main__":
    #pcd_data = o3d.data.DemoICPPointClouds()
    #source = o3d.io.read_point_cloud(pcd_data.paths[0])
    #source_final = copy.deepcopy(source)
    #target = o3d.io.read_point_cloud(pcd_data.paths[1])
    # Input point-clouds
    source = copy.deepcopy(pointSet1)
    target = copy.deepcopy(pointSet2)
    threshold = 0.105
    #trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
    #                         [-0.139, 0.967, -0.215, 0.7],
    #                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
    
    trans_init = np.asarray([[1, 0, 0, 0],
                             [0, 1, 0, 0],
                             [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
    


    #draw_registration_result_init(source, target, trans_init)
    #print("Initial alignment")

    evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, trans_init)
    
    evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, trans_init)
    
    evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, trans_init)

    print(evaluation, "\n")
    #point_to_point_icp(source, target, threshold, trans_init)
    point_to_plane_icp(source, target, threshold, trans_init)
    

    






26897
[ 178.79658907 -239.95599325  -23.43715061]
[ 178.79658907 -239.95599325  -23.43715061]
[-178.79658907 -239.95599325  -23.43715061]
Print the signed distances of the first 10 points
[0. 1. 1. ... 1. 1. 1.]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
RegistrationResult with fitness=2.230732e-04, inlier_rmse=6.010904e-02, and correspondence_set size of 6
Access transformation to get result. 

Apply point-to-plane ICP
RegistrationResult with fitness=2.230732e-04, inlier_rmse=3.437568e-02, and correspondence_set size of 6
Access transformation to get result.
Transformation is:
[[ 9.99999970e-01 -2.44210627e-04  5.49592943e-13  3.57457725e-02]
 [ 2.44210627e-04  9.99999970e-01  8.74277463e-17 -1.46125520e-02]
 [-5.49592948e-13  4.67886935e-17  1.00000000e+00 -1.74909893e-14]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 

Print the distances of the first 10 points
